In [1]:
print("test")

test


In [1]:
import os
from huggingface_hub import login
from datasets import load_dataset
import tiktoken

/home/matty/BabyJoey/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
hf_token = os.getenv("HF_TOKEN")
if hf_token:
    login(hf_token)
else:
    raise ValueError("Hugging Face token not set.")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/matty/.cache/huggingface/token
Login successful


In [16]:
# Load the dataset
dataset = load_dataset("SouthernCrossAI/Project_Gutenberg_Australia")

In [17]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Paragraph Text', 'cl100k_base'],
        num_rows: 286370
    })
    validation: Dataset({
        features: ['Paragraph Text', 'cl100k_base'],
        num_rows: 53694
    })
    test: Dataset({
        features: ['Paragraph Text', 'cl100k_base'],
        num_rows: 17899
    })
})

In [4]:
cl100k_base_column = dataset["train"]["cl100k_base"]  # Adjust split if needed
print(f"Number of sequences: {len(cl100k_base_column)}")

Number of sequences: 286370


In [5]:
# Flatten the tokens
all_tokens = [token for sequence in cl100k_base_column for token in sequence]
print(f"Total tokens: {len(all_tokens)}")

Total tokens: 62804547


In [6]:
# Define chunk size and overlap (stride)
context_window = 512
stride = 256

# Create overlapping chunks
chunks = [
    all_tokens[i:i + context_window]
    for i in range(0, len(all_tokens) - context_window + 1, stride)
]

print(f"Number of overlapping chunks: {len(chunks)}")
print(f"Example chunk: {chunks[0]}")


Number of overlapping chunks: 245329
Example chunk: [46639, 526, 433, 682, 1359, 1099, 26902, 23750, 3258, 11, 330, 2465, 15369, 5084, 311, 617, 83410, 757, 13, 1115, 4131, 315, 3515, 264, 3361, 49837, 369, 264, 5333, 13, 358, 1176, 65047, 28883, 264, 11102, 13272, 11, 1243, 74537, 11, 323, 1457, 31685, 2011, 6445, 264, 30077, 26, 323, 3686, 17354, 358, 2733, 439, 3582, 358, 1436, 6140, 264, 2763, 315, 3958, 15369, 369, 279, 25491, 315, 423, 34364, 14637, 5721, 13, 358, 649, 956, 1781, 1268, 433, 374, 279, 3828, 61191, 757, 779, 13, 358, 5895, 994, 358, 4985, 1518, 1077, 1578, 30, 2030, 1070, 596, 279, 3070, 11, 323, 279, 7126, 13, 3092, 39526, 0, 433, 1053, 387, 279, 38736, 596, 1866, 15369, 311, 617, 311, 1650, 26757, 14637, 5721, 364, 23881, 3502, 31412, 3238, 358, 8434, 956, 656, 433, 369, 279, 72021, 6548, 11, 477, 10437, 478, 23726, 11, 477, 294, 1673, 13744, 4579, 11, 477, 8369, 478, 4851, 430, 3596, 574, 13, 2360, 11, 28146, 11, 1070, 596, 912, 32961, 26, 279, 17983, 6420, 315,

In [13]:
# Convert chunks to a tensor
input_tensor = torch.tensor(chunks, dtype=torch.long)
print(f"Input tensor shape: {input_tensor.shape}")

Input tensor shape: torch.Size([245329, 512])


In [14]:
from torch.utils.data import TensorDataset, DataLoader

# Define the batch size
batch_size = 32

# Create a TensorDataset and DataLoader
dataset = TensorDataset(input_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Inspect the DataLoader
for batch in dataloader:
    print(f"Batch shape: {batch[0].shape}")  # Should print [batch_size, context_window]
    break  # Display one batch


Batch shape: torch.Size([32, 512])


In [15]:
# Iterate through the DataLoader
for i, batch in enumerate(dataloader):
    print(f"Batch {i+1}:")
    print(batch[0])  # Display the batch
    if i == 2:  # Show only the first 3 batches
        break


Batch 1:
tensor([[ 2884,  1690,   264,  ...,   555,   279,  8013],
        [  430,  1364,   574,  ..., 18098,  4613,    13],
        [  315,   279,  6134,  ..., 22225,   539,   617],
        ...,
        [ 8036,   323, 44278,  ..., 13463,    13,  1628],
        [   11, 94370,   311,  ...,   311,   279,  7173],
        [ 5425, 22327,   315,  ...,   315,   279, 15429]])
Batch 2:
tensor([[   11,  1606,   279,  ...,   355,    11, 20355],
        [  304,   813, 79340,  ...,   304,   813,  6548],
        [ 2170, 14733,   439,  ...,   315,  7878,  3805],
        ...,
        [65657,  1139,   813,  ...,   323,   649,  2646],
        [ 1047,  1027,  1766,  ...,   791, 68457,  2965],
        [   13,  6187,  4238,  ..., 21483,  5304,   279]])
Batch 3:
tensor([[16914,   279, 23709,  ...,  5811,   439,   311],
        [ 1518,   433,   682,  ...,    11, 38067,   922],
        [71015, 12678,  9621,  ...,  1866, 11401,    11],
        ...,
        [  477,  9660,   315,  ..., 12092, 11977,    11],
    

In [11]:
import tiktoken
tokenizer = tiktoken.get_encoding("cl100k_base")

# Decode the tokens
decoded_text = tokenizer.decode(chunks[2000])
print("Decoded Text:")
print(decoded_text)

Decoded Text:
 I knew him he had been permanently settled there for about fourteen years, during which time he had enjoyed the most perfect health.Talking of statues, there is often, from one point of view, an unforeseen effect which is not possible in pictures--a point of a cocked hat, for instance, which suggests a beak, or a rapier sticking out behind, and giving the figure a tail. There is in the statue of Lord Nelson, on a tall column in Trafalgar Square, an effect which is greatly admired by the Americans who patronize Morley's Hotel on the Strand side of the Square. There is a similar--or even more so--effect in the statue of Captain Cook in Sydney, seen from one point of view. It's strange that these things are never foreseen. The sculptors must have had a rough time amongst their friends."Well, Phoebe, as I told you before, you can't expect to pick and choose; and if you want to be a favourite you ought to be civil to everybody. Now, Charlie Baker will speak against you and gi

In [18]:
import os
from huggingface_hub import login
from datasets import load_dataset
import torch
from torch.utils.data import TensorDataset, DataLoader

# Login to Hugging Face Hub
hf_token = os.getenv("HF_TOKEN")
if hf_token:
    login(hf_token)
else:
    raise ValueError("Hugging Face token not set.")

# Load the dataset
dataset = load_dataset("SouthernCrossAI/Project_Gutenberg_Australia")

# Helper function to process a split and create chunks
def process_split(split, context_window=512, stride=256):
    # Extract the 'cl100k_base' column
    cl100k_base_column = dataset[split]["cl100k_base"]
    print(f"Number of sequences in {split}: {len(cl100k_base_column)}")
    
    # Flatten the tokens
    all_tokens = [token for sequence in cl100k_base_column for token in sequence]
    print(f"Total tokens in {split}: {len(all_tokens)}")
    
    # Create overlapping chunks
    chunks = [
        all_tokens[i:i + context_window]
        for i in range(0, len(all_tokens) - context_window + 1, stride)
    ]
    print(f"Number of overlapping chunks in {split}: {len(chunks)}")
    return torch.tensor(chunks, dtype=torch.long)

# Process training and validation splits
context_window = 512
stride = 256
train_tensor = process_split("train", context_window, stride)
val_tensor = process_split("validation", context_window, stride)

# Create TensorDatasets
train_dataset = TensorDataset(train_tensor)
val_dataset = TensorDataset(val_tensor)

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Inspect the DataLoaders
print("Training DataLoader:")
for batch in train_loader:
    print(f"Batch shape: {batch[0].shape}")  # Should print [batch_size, context_window]
    break

print("Validation DataLoader:")
for batch in val_loader:
    print(f"Batch shape: {batch[0].shape}")  # Should print [batch_size, context_window]
    break


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/matty/.cache/huggingface/token
Login successful
Number of sequences in train: 286370
Total tokens in train: 62804547
Number of overlapping chunks in train: 245329
Number of sequences in validation: 53694
Total tokens in validation: 11709666
Number of overlapping chunks in validation: 45739
Training DataLoader:
Batch shape: torch.Size([32, 512])
Validation DataLoader:
Batch shape: torch.Size([32, 512])


In [29]:
sample = next(iter(val_loader))

In [41]:
print(sample[0].tolist()[0])

[22336, 527, 1457, 73734, 449, 279, 7095, 358, 1053, 33330, 5304, 499, 25, 47177, 311, 757, 11, 8613, 78, 11, 430, 499, 690, 1304, 420, 16540, 369, 757, 311, 3842, 423, 6172, 323, 311, 912, 1023, 3823, 1694, 0, 220, 4593, 686, 433, 555, 279, 3021, 499, 3131, 1071, 499, 39786, 757, 9135, 220, 3005, 65330, 1203, 39019, 323, 79578, 856, 2077, 13, 1789, 264, 4545, 358, 76035, 539, 7095, 7182, 311, 6604, 11, 3686, 2555, 2011, 387, 1071, 13, 220, 1666, 358, 10555, 1077, 3242, 266, 1873, 358, 20592, 25, 330, 43, 6863, 11, 499, 617, 30831, 264, 2294, 4785, 505, 856, 4851, 323, 9277, 264, 32415, 832, 5304, 433, 13, 1789, 47530, 757, 430, 358, 617, 3596, 93762, 499, 13, 220, 7570, 439, 499, 617, 1027, 837, 311, 6261, 11, 358, 47177, 555, 279, 3021, 358, 2103, 11984, 499, 311, 6493, 701, 1984, 311, 423, 6172, 43059, 581, 323, 311, 912, 1023, 3823, 1694, 13, 220, 358, 4985, 5379, 701, 4339, 520, 3131, 311, 4477, 430, 4400, 1253, 387, 5675, 1555, 279, 8060, 315, 856, 5044, 1210, 1687, 12570, 304, 8

In [40]:
import tiktoken

# Initialize the tokenizer
tokenizer = tiktoken.get_encoding("cl100k_base")

# Get a sample batch
sample = next(iter(val_loader))

# Convert the batch (tensor) to a list of sequences
batch_tokens = sample[0].tolist()  # Each sublist is a sequence

# Decode each sequence
decoded_texts = [tokenizer.decode(tokens) for tokens in batch_tokens]

# Print the decoded texts
for i, text in enumerate(decoded_texts):
    print(f"Decoded Text {i+1}:")
    print(text)
    print()


Decoded Text 1:
"You are now acquainted with the trust I would impose upon you: swear to me, Moro, that you will make this explanation for me to John Darrow and to no other human being!  Swear it by the love you once said you bore me!"  She sank back exhausted and awaited my response. For a moment I dared not trust myself to speak, yet something must be said.  As I noted her impatience I replied: "Lona, you have lifted a great weight from my heart and placed a lesser one upon it. Forgive me that I have ever doubted you.  Even as you have been true to yourself, I swear by the love I still bear you to deliver your message to Darrow Sahib and to no other human being.  I shall commit your words at once to writing that nothing may be lost through the failure of my memory."We spoke in French, for I had discovered that this was her favourite speech.  There was no Oliphant to wait on us, and the inn servants were always about, so it was well to have a tongue they did not comprehend. The lady w